# Bloch-Torrey Equation

## Introduction

Here we solve the Bloch-Torrey equation on a unit square, with the diffusion coefficient $D(x)$, relaxation rate $R(x)$, and resonance frequency $\omega(x)$ all given as a generic functions.
The strong form of the Bloch-Torrey equation is given by

\begin{align}
    \frac{\partial u_x}{\partial t} &= \nabla \cdot (D \nabla u_x) - R u_x + \omega u_y  \quad x \in \Omega\\
    \frac{\partial u_y}{\partial t} &= \nabla \cdot (D \nabla u_y) - R u_y - \omega u_x  \quad x \in \Omega,
\end{align}

where $\vec{u}=[u_x,u_y]$ is the transverse magnetization, and $\Omega$ the domain.

We will consider homogeneous Neumann boundary conditions such that

\begin{align}
    \nabla \vec{u}(x) \cdot \hat{n} &= \vec{0}  \quad x \in \partial \Omega\\
\end{align}

where $\partial \Omega$ denotes the boundary of $\Omega$, and $\cdot$ is a tensor contraction.

The initial condition is given generically as

\begin{equation}
    \vec{u}(x,t=0) = \vec{u}_0 (x)  \quad x \in \Omega
\end{equation}


The resulting weak form is given by
\begin{align}
    \int_{\Omega} \vec{v} \cdot \vec{u}_t \, d\Omega
    &= -\int_{\Omega}
    -\vec{v} \cdot \nabla \cdot ( D \, \nabla \vec{u} ) +
    R \, \vec{v} \cdot \vec{u} -
    \omega \, \vec{v} \times \vec{u}
    \, d\Omega \\
    &= -\int_{\Omega}
    D \, \nabla \vec{v} : \nabla \vec{u} +
    R \, \vec{v} \cdot \vec{u} -
    \omega \, \vec{v} \times \vec{u}
    \, d\Omega + 
    \int_{\partial\Omega} \vec{v} \cdot (D\nabla\vec{u} \cdot \hat{n}) \, d\Gamma,
\end{align}
where $\vec{v}$ is a suitable test function.

In this notebook, we will assume homogeneous Neumann boundary conditions on all boundaries by taking $D\nabla\vec{u} \cdot \hat{n} = 0$. Therefore, the final weak form is simply
\begin{align}
    \int_{\Omega} \vec{v} \cdot \vec{u}_t \, d\Omega
    = -\int_{\Omega}
    D \, \nabla \vec{v} : \nabla \vec{u} +
    R \, \vec{v} \cdot \vec{u} -
    \omega \, \vec{v} \times \vec{u}
    \, d\Omega
\end{align}

Note: in two dimensions, the cross product is simply a scalar. However, `Tensors.jl` defines the two dimensional cross product by first extending the 2D vectors into 3D. Below, we use the symbol $\boxtimes$ to denote the scalar version, which is the same as taking the third component of the vector version

## Commented Program

Now we solve the problem in JuAFEM. What follows is a program spliced with comments.

First we load the JuAFEM package.

In [1]:
# using Distributed
# addprocs(8; restrict = true, enable_threaded_blas = true);
# @everywhere begin
#     include("init.jl")
# end

In [151]:
include("init.jl")

**Pack circles**: Pack circles with a specified packing density $\eta$

In [152]:
btparams = BlochTorreyParameters{Float64}(
    theta = pi/2,
    AxonPDensity = 0.83,
    g_ratio = 0.8,
    D_Tissue = 25, # [μm²/s]
    D_Sheath = 25, # [μm²/s]
    D_Axon = 25 # [μm²/s]
);

In [150]:
TE_typical = 10e-3
nDim = 2
dist_typical = (2π * btparams.R_mu)/4
time_typical = TE_typical/2
D_Maximal_Dephasing = dist_typical^2/(2 * nDim * time_typical)

26.105103640881353

In [90]:
Dim = 2;
Ncircles = 30;

η = btparams.AxonPDensity; # goal packing density
ϵ = 0.1 * btparams.R_mu; # overlap occurs when distance between circle edges is ≤ ϵ
α = 1e-1; # covariance penalty weight (enforces circular distribution)
β = 1e-6; # mutual distance penalty weight
λ = 1.0; # overlap penalty weight (or lagrange multiplier for constrained version)

# rs = rand(radiidistribution(btparams), Ncircles);
# initial_circles = GreedyCirclePacking.pack(rs; goaldensity = η + 0.01, iters = 10000)
@show estimate_density(initial_circles)

estimate_density(initial_circles) = 0.8010047898161851


0.8010047898161851

In [70]:
@show minimum_signed_edge_distance(initial_circles);

minimum_signed_edge_distance(initial_circles) = -4.440892098500626e-16


In [71]:
@show estimate_density(initial_circles);

estimate_density(initial_circles) = 0.8010047898161851


In [7]:
@time outer_circles = EnergyCirclePacking.pack(initial_circles;
    autodiff = false,
    secondorder = false,
    setcallback = false,
    goaldensity = η,
    distancescale = btparams.R_mu,
    weights = [α, β, λ],
    epsilon = ϵ
);
inner_circles = scale_shape.(outer_circles, btparams.g_ratio);

  7.784137 seconds (14.70 M allocations: 729.498 MiB, 11.77% gc time)


┌ Warning: Density cannot be reached without overlapping circles; can only reach 0.802667243955799 < 0.83
└ @ CirclePackingUtils /home/coopar7/Documents/code/BlochTorreyExperiments-master/Experiments/MyelinWaterOrientation/CirclePacking/CirclePackingUtils.jl:103


In [8]:
dmin = minimum_signed_edge_distance(outer_circles)
@show covariance_energy(outer_circles)
@show estimate_density(outer_circles)
@show is_any_overlapping(outer_circles)
@show (dmin, ϵ, dmin > ϵ);

covariance_energy(outer_circles) = 0.05843524979929998
estimate_density(outer_circles) = 0.802667243955799
is_any_overlapping(outer_circles) = false
(dmin, ϵ, dmin > ϵ) = (2.0872192862952943e-14, 0.046000000000000006, false)


**Generate mesh**: Rectangular mesh with circles possibly only partly contained or completely excluded

In [9]:
#Ncircles = length(outer_circles);
#Nmin = 50; # points for average circle
#h0 = 2pi*R_mu/Nmin; # approximate scale
h0 = minimum(radius.(outer_circles))*(1-btparams.g_ratio); # fraction of size of minimum torus width
h_min = 1.0*h0; # minimum edge length
h_max = 5.0*h0; # maximum edge length
h_range = 10.0*h0; # distance over which h increases from h_min to h_max
h_rate = 0.6; # rate of increase of h from circle boundaries (power law; smaller = faster radial increase)

0.6

In [10]:
bdry, _ = opt_subdomain(outer_circles);

In [11]:
@time exteriorgrids, torigrids, interiorgrids, parentcircleindices = disjoint_rect_mesh_with_tori(
    bdry, inner_circles, outer_circles, h_min, h_max, h_range, h_rate;
    maxstalliters = 10000, plotgrids = false, exterior_tiling = (2, 2)
);

1/30: Interior
1/30: Annular
2/30: Interior
2/30: Annular
3/30: Interior
3/30: Annular
4/30: Interior
4/30: Annular
5/30: Interior
5/30: Annular
6/30: Interior
6/30: Annular
7/30: Interior
7/30: Annular
8/30: Interior
8/30: Annular
9/30: Interior
9/30: Annular
10/30: Interior
10/30: Annular
11/30: Interior
11/30: Annular
12/30: Interior
12/30: Annular
13/30: Interior
13/30: Annular
14/30: Interior
14/30: Annular
15/30: Interior
15/30: Annular
16/30: Interior
16/30: Annular
17/30: Interior
17/30: Annular
18/30: Interior
18/30: Annular
19/30: Interior
19/30: Annular
20/30: Interior
20/30: Annular
21/30: Interior
21/30: Annular
22/30: Interior
22/30: Annular
23/30: Interior
23/30: Annular
24/30: Interior
24/30: Annular
25/30: Interior
25/30: Annular
26/30: Interior
26/30: Annular
27/30: Interior
27/30: Annular
28/30: Interior
28/30: Annular
29/30: Interior
29/30: Annular
30/30: Interior
30/30: Annular
1/4: Exterior
2/4: Exterior
3/4: Exterior
4/4: Exterior
1359.162482 seconds (10.88 G all

In [12]:
simpplot(vcat(exteriorgrids, torigrids, interiorgrids); newfigure = true, axis = mxaxis(bdry));

### Diffusion coefficient $D(x)$, relaxation rate $R(x)$, and resonance frequency $\omega(x)$

These functions are defined within `doassemble!`

### Trial and test functions
A `CellValues` facilitates the process of evaluating values and gradients of
test and trial functions (among other things). Since the problem
is a scalar problem we will use a `CellScalarValues` object. To define
this we need to specify an interpolation space for the shape functions.
We use Lagrange functions (both for interpolating the function and the geometry)
based on the reference "cube". We also define a quadrature rule based on the
same reference cube. We combine the interpolation and the quadrature rule
to a `CellScalarValues` object.

### Degrees of freedom
Next we need to define a `DofHandler`, which will take care of numbering
and distribution of degrees of freedom for our approximated fields.
We create the `DofHandler` and then add a single field called `u`.
Lastly we `close!` the `DofHandler`, it is now that the dofs are distributed
for all the elements.

Now that we have distributed all our dofs we can create our tangent matrix,
using `create_sparsity_pattern`. This function returns a sparse matrix
with the correct elements stored.

We can inspect the pattern using the `spy` function from `UnicodePlots.jl`.
By default the stored values are set to $0$, so we first need to
fill the stored values, e.g. `K.nzval` with something meaningful.

In [13]:
#using UnicodePlots
#fill!(K.nzval, 1.0)
#spy(K; height = 25)

### Boundary conditions
In JuAFEM constraints like Dirichlet boundary conditions are handled by a `ConstraintHandler`. However, here we will have no need to directly enforce boundary conditions, since Neumann boundary conditions have already been applied in the derivation of the weak form.

### Assembling the linear system
Now we have all the pieces needed to assemble the linear system, $K u = f$.
We define a function, `doassemble` to do the assembly, which takes our `cellvalues`,
the sparse matrix and our DofHandler as input arguments. The function returns the
assembled stiffness matrix, and the force vector.

In [153]:
myelinprob = MyelinProblem(btparams);

In [154]:
myelindomains = createmyelindomains(exteriorgrids, torigrids, interiorgrids, outer_circles, inner_circles);

In [157]:
# @time doassemble!(prob, domains);
@time map!(m -> doassemble!(m, myelinprob), myelindomains, myelindomains);

  3.510104 seconds (58.11 M allocations: 1.122 GiB, 13.11% gc time)


In [158]:
# @time factorize!(domains);
@time map!(m -> (factorize!(getdomain(m)); return m), myelindomains, myelindomains);

  0.086534 seconds (23.29 k allocations: 24.789 MiB, 13.13% gc time)


### Plotting the resonance frequency map $\omega(x)$
$\omega(x)$ for each region can be easily created by accessing the `Omega` field of a `BlochTorreyProblem` object. Now, evaluate $\omega(x)$ on each node `x` and plot the resuling field map overtop of the tesselation.

In [159]:
omegavalues = map(myelindomains) do m
    ω = BlochTorreyProblem(myelinprob, m).Omega
    return map(getnodes(getgrid(m))) do node
        ω(getcoordinates(node))
    end
end;

In [160]:
simpplot(getgrid.(myelindomains); newfigure = true, axis = mxaxis(bdry), facecol = reduce(vcat, omegavalues));

### Solution of the differential equation system
The last step is to solve the system. First we call `doassemble`
to obtain the global stiffness matrix `K` and force vector `f`.
Then, to account for the boundary conditions, we use the `apply!` function.
This modifies elements in `K` and `f` respectively, such that
we can get the correct solution vector `u` by using `\`.

In [161]:
tspan = (0.0, 320.0e-3);
dt = 10e-3;
ts = tspan[1]:dt:tspan[2]
# saveat = tspan[1]:dt:tspan[2];
# tstops = (tspan[1] .+ dt/2 .+ dt .* (1:round(Int, (tspan[2]-tspan[1])/dt)))
u0 = Vec{2}((0.0, 1.0)); # initial pi/2 pulse

In [162]:
probs = [ODEProblem(m, interpolate(u0, m), tspan) for m in myelindomains];

In [163]:
sols = Vector{ODESolution}(undef, length(probs));

In [164]:
@time sols = map!(sols, 1:length(probs), probs) do i, prob
    print("i = $i/$(length(sols)): ")
    A = prob.p[1]
    return @time solve(prob, ExpokitExpmv(A; m = 30);
        dt = dt,
        reltol = 1e-4,
        callback = MultiSpinEchoCallback(tspan; TE = dt)
    )
end;

i = 1/62:   2.118724 seconds (211.24 k allocations: 418.809 MiB, 5.03% gc time)
i = 2/62:   2.988332 seconds (252.60 k allocations: 734.955 MiB, 5.11% gc time)
i = 3/62:   2.129449 seconds (213.41 k allocations: 457.417 MiB, 4.64% gc time)
i = 4/62:   2.485487 seconds (258.84 k allocations: 587.808 MiB, 5.14% gc time)
i = 5/62:   2.349258 seconds (222.15 k allocations: 566.108 MiB, 4.96% gc time)
i = 6/62:   0.450620 seconds (98.39 k allocations: 104.221 MiB, 5.15% gc time)
i = 7/62:   2.831512 seconds (222.85 k allocations: 551.087 MiB, 14.24% gc time)
i = 8/62:   2.410513 seconds (231.73 k allocations: 505.563 MiB, 4.33% gc time)
i = 9/62:   0.882491 seconds (138.45 k allocations: 186.760 MiB, 3.93% gc time)
i = 10/62:   1.097396 seconds (156.37 k allocations: 232.999 MiB, 4.29% gc time)
i = 11/62:   1.830065 seconds (213.09 k allocations: 406.184 MiB, 4.41% gc time)
i = 12/62:   2.526347 seconds (237.03 k allocations: 561.836 MiB, 4.90% gc time)
i = 13/62:   1.616181 seconds (227.99

In [186]:
Signals = map(myelindomains, sols) do m, s
    [integrate(s(t), m) for t in tspan[1]:dt:tspan[2]]
end;

In [187]:
Stotal = sum(Signals);

In [168]:
Tspan = tspan[2] - tspan[1]
@show btparams.R2_lp;
@show (-1/Tspan)*log(norm(Stotal[end])/norm(Stotal[1]));
@show exp(-tspan[end]*btparams.R2_lp);
@show norm(Stotal[end])/norm(Stotal[1]);

btparams.R2_lp = 15.873015873015873
(-1 / Tspan) * log(norm(Stotal[end]) / norm(Stotal[1])) = 16.955888300355195
exp(-(tspan[end]) * btparams.R2_lp) = 0.006223859418487457
norm(Stotal[end]) / norm(Stotal[1]) = 0.004401172676796397


In [169]:
exact_mwf = getmwf(outer_circles, inner_circles, bdry);
@show exact_mwf;

exact_mwf = 0.29074197131665613


In [170]:
mxcall(:addpath, 0, "/home/coopar7/Documents/code/BlochTorreyExperiments-master/Experiments/MyelinWaterOrientation/MATLAB/SE_corr/");

In [182]:
MWImaps, MWIdist, MWIpart = fitmwfmodel(Stotal, NNLSRegression();
    T2Range = [10e-3, 2.0],
    spwin = [10e-3, 40e-3],
    mpwin = [40.5e-3, 200e-3],
    nT2 = 32,
    RefConAngle = 165.0,
    PLOTDIST = true
);
getmwf(NNLSRegression(), MWImaps, MWIdist, MWIpart)

0.27250259997614135

In [172]:
# TestStotalMagn = mwimodel(TwoPoolMagnToMagn(), tspan[1]:dt:tspan[2], modelfit.param)
# TestStotal = [Vec{2}((zero(y),y)) for y in TestStotalMagn];

In [173]:
for modeltype in (TwoPoolMagnToMagn(), ThreePoolMagnToMagn(), ThreePoolCplxToMagn(), ThreePoolCplxToCplx())
    local modelfit, errors, mwf
    println("Model: $modeltype"); flush(stdout)
    modelfit, errors = fitmwfmodel(Stotal, modeltype; TE = dt);
    mwf = getmwf(modeltype, modelfit, errors)
    println("mwf: $mwf"); flush(stdout)
    errors == nothing ? display(modelfit.param) : display([modelfit.param errors]); flush(stdout)
end

Model: TwoPoolMagnToMagn()
mwf: 0.2808416133749954


4×2 Array{Float64,2}:
  5.74069  0.00321124
 14.7003   0.0039422 
 67.4581   0.0809893 
 15.8815   0.00252236

Model: ThreePoolMagnToMagn()
mwf: 0.2805980690215158


6×2 Array{Float64,2}:
  5.7361      0.0286866
 10.0701   1045.98     
  4.63621  1045.95     
 67.5229      0.271054 
 15.7337     24.5262   
 16.2238     57.4327   

Model: ThreePoolCplxToMagn()
mwf: 0.2580223204316522


8×2 Array{Float64,2}:
  5.25719     1.21187
 10.5225     98.7384 
  4.59528    99.7441 
 68.6898     12.4927 
 15.1608      7.46731
 18.8163     59.2509 
  2.63837    64.5973 
  0.0365539  91.7937 

Model: ThreePoolCplxToCplx()
mwf: 0.279705249390804


10×2 Array{Float64,2}:
  5.71938  0.0927131  
 13.4139   2.93551    
  1.31457  3.02249    
 67.8159   0.945698   
 15.5945   0.386491   
 18.9859   2.43119    
 50.3446   0.0619796  
 49.9828   0.0419578  
 50.6108   1.02431    
 -1.5717   0.000867294

In [174]:
widths(bdry)

2-element Tensor{1,2,Float64,2}:
 4.597908069979724
 4.597908069979724

In [175]:
@show getmwf(outer_circles, inner_circles, bdry)
@show getmwf(Stotal, TwoPoolMagnToMagn(); TE = dt, fitmethod = :local);
@show getmwf(Stotal, ThreePoolMagnToMagn(); TE = dt, fitmethod = :local);
@show getmwf(Stotal, ThreePoolCplxToMagn(); TE = dt, fitmethod = :local);
@show getmwf(Stotal, ThreePoolCplxToCplx(); TE = dt, fitmethod = :local);

getmwf(outer_circles, inner_circles, bdry) = 0.29074197131665613
getmwf(Stotal, TwoPoolMagnToMagn(); TE=dt, fitmethod=:local) = 0.2808416133749954
getmwf(Stotal, ThreePoolMagnToMagn(); TE=dt, fitmethod=:local) = 0.2805980690215158
getmwf(Stotal, ThreePoolCplxToMagn(); TE=dt, fitmethod=:local) = 0.2580223204316522
getmwf(Stotal, ThreePoolCplxToCplx(); TE=dt, fitmethod=:local) = 0.279705249390804


In [176]:
p0 = initialparams(ThreePoolCplxToCplx(), ts, Stotal)[1];
modelfit, errors = fitmwfmodel(Stotal, ThreePoolCplxToCplx(); TE = dt);

In [177]:
# mwimodel(ThreePoolCplxToCplx(), ts, modelfit.param);
# [mwimodel(ThreePoolCplxToCplx(), ts,  modelfit.param) |> x -> reinterpret(ComplexF64, x) complex.(Stotal)]
# [mwimodel(ThreePoolCplxToCplx(), ts, p0) |> x -> reinterpret(ComplexF64, x) complex.(Stotal)]

In [178]:
myelin_area = intersect_area(outer_circles, bdry) - intersect_area(inner_circles, bdry)
total_area = area(bdry)
y_biexp = @. (total_area - myelin_area) * exp(-ts*btparams.R2_lp) + myelin_area * exp(-ts*btparams.R2_sp);

In [183]:
mxcall(:figure, 0)
mxcall(:plot, 0, collect(1000.0.*ts), [norm.(Stotal) y_biexp])

In [184]:
mxcall(:legend, 0, "Simulated", "Bi-Exponential")
mxcall(:title, 0, "Signal Magnitude vs. Time")
mxcall(:xlabel, 0, "Time [ms]"); mxcall(:xlim, 0, 1000.0 .* [tspan...])
mxcall(:ylabel, 0, "S(t) Magnitude")

In [191]:
mxcall(:figure, 0)
mxcall(:plot, 0, collect(1000.0.*ts), reduce(hcat, map(S->norm.(S), Signals)))

In [39]:
# prob = ODEProblem((du,u,p,t)->A_mul_B!(du,p[1],u), u0, tspan, (Amap,));

In [40]:
#@time Expokit.expmv!(u, tspan[end], Amap, u0; tol=1e-4, norm=expmv_norm, m=100); # penelope: 17.42s
#@time Expokit.expmv!(u, tspan[end], Amap, u0; tol=1e-4, norm=expmv_norm, m=50); # penelope: 30.09s
#@time Expokit.expmv!(u, tspan[end], Amap, u0; tol=1e-4, norm=expmv_norm, m=10); # penelope: 103.5s
#@time Expokit.expmv!(u, tspan[end], Amap, u0; tol=1e-8, norm=expmv_norm); # penelope: 53.2s
#@time Expokit.expmv!(u, tspan[end], Amap, u0; tol=1e-6, norm=expmv_norm); # penelope: 44.4s

In [41]:
#@time sol = solve(prob, CVODE_BDF(linear_solver=:GMRES); saveat=tspan, reltol=1e-8, alg_hints=:stiff); # penelope: 90.21s
#@time sol = solve(prob, CVODE_BDF(linear_solver=:GMRES); saveat=tspan, reltol=1e-4, alg_hints=:stiff); # penelope: 33.44s
#@time sol = solve(prob, CVODE_BDF(linear_solver=:BCG); saveat=tspan, reltol=1e-4, alg_hints=:stiff) # penelope: 53.66s
#@time sol = solve(prob, CVODE_BDF(linear_solver=:TFQMR); saveat=tspan, reltol=1e-4, alg_hints=:stiff) # penelope: 18.99s but low accuracy

In [42]:
#prob_Ku = ODEProblem(K_mul_u!, u0, tspan, (K,), mass_matrix=M);
#@time sol_Ku = solve(prob_Ku, Rosenbrock23(), saveat=tspan, reltol=1e-4, alg_hints=:stiff) #DNF
#@time sol_Ku = solve(prob_Ku, Rodas4(), saveat=tspan, reltol=1e-4, alg_hints=:stiff) #DNF

In [43]:
# @show norm(sol.u[end] - u)/maximum(abs,u);
# @show maximum(sol.u[end] - u)/maximum(abs,u);

# Testing

In [ ]:
using BlochTorreyUtilsTest

### Single Axon

In [ ]:
BlochTorreyUtilsTest.singleaxontests(
    BlochTorreyParameters{Float64}(
        ChiI = -60e-9,
        ChiA = -120e-9
    );
    PLOTOMEGA = false
);

### Multiple Axons

In [ ]:
domainsetup = BlochTorreyUtilsTest.multipleaxons();

In [ ]:
BlochTorreyUtilsTest.multipleaxontests(
    BlochTorreyParameters{Float64}(
        ChiI = -60e-9,
        ChiA = -120e-9
    ),
    domainsetup;
    PLOTOMEGA = false
);

### Exporting to VTK
To visualize the result we export the grid and our field `u`
to a VTK-file, which can be viewed in e.g. [ParaView](https://www.paraview.org/).

In [ ]:
# vtk_grid("bloch_torrey_equation", dh) do vtk
#     vtk_point_data(vtk, dh, u)
# end